# Fine Tunning de YOLOv8 utilizando GroundingDINO para etiquetar




**Importante:** primero tienen que activar aceleración con GPU. Ir a Editar -> Configuración del notebook -> Seleccionar T4GPU en acelerador.

In [ ]:
import os
from IPython.display import Image, display

# Clonamos repo de GroundingDINO:
!git clone https://github.com/IDEA-Research/GroundingDINO.git

HOME = os.getcwd()
%cd {HOME}/GroundingDINO
!pip install -r requirements.txt
!pip install -q -e .

# Descargamos pesos entrenados:
! wget -q https://github.com/IDEA-Research/GroundingDINO/releases/download/v0.1.0-alpha/groundingdino_swint_ogc.pth

# Descargamos imagen de demo:
!mkdir tomatoes_dataset
!wget -q wget -q "https://drive.usercontent.google.com/download?id=1vbHqup5TL1cnm_iefyq5aDUA3lBDoQMh&export=download" -O tomatoes_dataset/tomatoes1.jpeg


## Grounding DINO

In [ ]:
#Ejemplo de una predicción de GroundingDINO en una imagen de tomates

!python3 demo/inference_on_a_image.py \
    --config_file 'groundingdino/config/GroundingDINO_SwinT_OGC.py' \
    --checkpoint_path 'groundingdino_swint_ogc.pth' \
    --image_path 'tomatoes_dataset/tomatoes1.jpeg'  \
    --text_prompt 'tomato' \
    --box_threshold 0.35 \
    --text_threshold 0.01 \
    --output_dir 'outputs'

In [ ]:
display(Image(filename='outputs/pred.jpg'))

In [ ]:
# Vamos a ver un segundo ejemplo con autos. Guardamos la imagen con el nombre `autos.jpg`
!wget https://c7.alamy.com/compes/g6jefn/muchos-coches-color-sobre-un-fondo-blanco-hecho-en-3d-rendering-g6jefn.jpg -O autos.jpg
display(Image(filename='autos.jpg'))

Prueben ejecutando GroundingDINO sobre la imagen de autos y cambiando el text_prompt.

In [ ]:
#TODO: completar para la predicción en la imagen de autos, probar diferentes prompts: cómo cambia con 'car', 'red car' y 'red car going forward'
#TODO: encontrar el path a la imagen `autos.jpg`

!python3 demo/inference_on_a_image.py \
    --config_file 'groundingdino/config/GroundingDINO_SwinT_OGC.py' \
    --checkpoint_path 'groundingdino_swint_ogc.pth' \
    --image_path '...'  \
    --text_prompt '...' \
    --box_threshold 0.35 \
    --text_threshold 0.01 \
    --output_dir 'outputs'

display(Image(filename='outputs/pred.jpg'))

In [ ]:
!pip install fiftyone
!pip install typer

In [ ]:
#Cargamos el modelo con los pesos para generar labels en el dataset de comida.

from groundingdino.util.inference import load_model, load_image, predict, annotate

def run_dino(dino, image, text_prompt='food', box_threshold=0.4, text_threshold=0.1):
    boxes, logits, phrases = predict(
        model = dino,
        image = image,
        caption = text_prompt,
        box_threshold = box_threshold,
        text_threshold = text_threshold
    )
    return boxes, logits, phrases

dino = load_model('groundingdino/config/GroundingDINO_SwinT_OGC.py', 'groundingdino_swint_ogc.pth')

### Creamos las etiquetas en el dataset

In [ ]:
def annotate(dino, data, data_size, data_dir):

    image_dir = f'{data_dir}/images'
    label_dir = f'{data_dir}/labels'
    os.makedirs(image_dir, exist_ok=True)
    os.makedirs(label_dir, exist_ok=True)

    for i, d in enumerate(data):
        image_path = f'{image_dir}/{i:06d}.png'
        label_path = f'{label_dir}/{i:06d}.txt'
        image = d['image'].resize((640, 640))
        image.save(image_path)

        image_source, image = load_image(image_path)
        boxes, logits, phrases = run_dino(dino, image)

        label = ['0 ' + ' '.join(list(map(str, b))) for b in boxes.tolist()]
        label = '\n'.join(label)
        with open(label_path, 'w') as f:
            f.write(label)

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset


train_data = load_dataset('food101', split='train[:3000]')

val_data = load_dataset('food101', split='validation[:1000]')

data = {"train": train_data, "validation": val_data}


Las siguientes celdas crean las labels para train y test.

Si está tardando mucho pueden frenar la ejecución después de un par de minutos en lugar de etiquetar todo el dataset.

In [ ]:
annotate(dino, data['train'], 3000, 'data/train') #Anotación de labels para train

In [ ]:
annotate(dino, data['validation'], 1000, 'data/valid') #Anotación de labels para validación

## YOLO
Entrenamos el modelo YOLO con las anotaciones de GroundDINO

In [ ]:
!pip install ultralytics

In [ ]:
from ultralytics import YOLO

Para el entrenamiento, tenemos que hacer un archivo de configuración. En este archivo le decimos las categorías que vamos a predecir y la ruta hacia train y validación.

In [ ]:
import yaml
config = {
    'names': ['food'],
    'nc': 1,
    'train': '/content/GroundingDINO/data/train/images',
    'val': '/content/GroundingDINO/data/valid/images'
}

with open('data/data.yaml', 'w') as f:
    yaml.dump(config, f)

Entrenamos sobre los pesos de YOLO con el método `yolo.train()`.

In [ ]:
yolo = YOLO("yolov8n.pt")
yolo.train(data="data/data.yaml", epochs=10)
valid_results = yolo.val()
print(valid_results)

### Comparación de nuestro modelo

Vamos a comparar nuestro modelo tuneado contra YOLO normal.

Les deje 4 imagenes de interner para que prueben y analicen la diferencia entre el modelo original y nuestro tuneado. ¿Qué diferencias encuentran?

In [ ]:
from ultralytics import YOLO
from PIL import Image as Img
from PIL import ImageTk

# Cargamos los dos modelos
modelo_original = YOLO('yolov8n.pt')
modelo_tuneado = YOLO('/content/GroundingDINO/runs/detect/train/weights/best.pt')

imagen = 'https://miro.medium.com/v2/resize:fit:1100/format:webp/1*VzvFyoTUMecfSyEQZ3fqDg.png'
#imagen = 'https://ae01.alicdn.com/kf/Saab8207bd19f4ddaa559cdd6bf19a50fG.jpeg_640x640Q90.jpeg_.webp'
#imagen = 'https://previews.123rf.com/images/smaglov/smaglov1206/smaglov120600910/14163853-bandeja-de-comida-en-el-avi%C3%B3n.jpg'
#imagen = 'https://www.shutterstock.com/image-photo/minimal-fast-food-tray-top-600nw-2290762729.jpg'

resultado_1 = modelo_original(imagen)
resultado_2 = modelo_tuneado(imagen)

for i, r in enumerate(resultado_1):
    im_bgr = r.plot()
    im_rgb = Img.fromarray(im_bgr[..., ::-1])

    # Guardar resultados
    r.save(filename='resultado_1.jpg')

for i, r in enumerate(resultado_2):
    im_bgr = r.plot()
    im_rgb = Img.fromarray(im_bgr[..., ::-1])

    # Guardar resultados
    r.save(filename='resultado_2.jpg')

In [ ]:
from IPython.display import Image, display
display(Image(filename='resultado_1.jpg'))
display(Image(filename='resultado_2.jpg'))



**Ejercicio:** buscar otra foto en internet en donde el modelo tuneado detecte comida y el modelo original detecte otras cosas.